In [1]:
import intake
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xesmf as xe
import pandas as pd
import zarr
import gcsfs
import requests
import warnings
warnings.filterwarnings("ignore")
##### Cloud data
url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
gcs = gcsfs.GCSFileSystem(token='anon')
df  = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

AttributeError: Unknown open method 'open_esm_datastore'. Do you need to install a new driver from the plugin directory? https://intake.readthedocs.io/en/latest/plugin-directory.html
Registered opener methods: ['open_alias', 'open_catalog', 'open_csv', 'open_intake_remote', 'open_json', 'open_jsonl', 'open_ndzarr', 'open_numpy', 'open_textfiles', 'open_tiled', 'open_tiled_cat', 'open_yaml_file_cat', 'open_yaml_files_cat', 'open_zarr_cat', 'open_netcdf', 'open_opendap', 'open_rasterio', 'open_remote-xarray', 'open_xarray_image', 'open_zarr']

In [ ]:
def read_data(im):
    im=im
    ie='abrupt-4xCO2'
    iv='tos'
    cat = col.search(
        source_id=im,
        experiment_id=ie,
        variable_id=iv,
        table_id='Omon',
        grid_label='gn',
        member_id='r1i1p1f1',
    )
    dset_dict = cat.to_dataset_dict(zarr_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True})
    ds = dset_dict[list(dset_dict.keys())[0]].isel(time=0)
    return ds

In [ ]:
im = 'CanESM5'
ds = read_data(im).squeeze()
grid_in   = {'lon': ds['tos']['longitude'], 'lat': ds['tos']['latitude']} 
grid_out  = {'lon': np.linspace(0, 359, 360), 'lat': np.linspace(-89.5, 89.5, 180)}
regridder = xe.Regridder(grid_in, grid_out, 'bilinear', periodic=True, ignore_degenerate=True)
tos_regrid           = regridder(ds['tos'])
tos_regrid_ignore_NA = regridder(ds['tos'], skipna=True)

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(18,3))
axs      = axs.flatten()
ds['tos'].plot(ax=axs[0])            ; axs[0].set_title(im+' raw tos');
tos_regrid.plot(ax=axs[1])           ; axs[1].set_title(im+' tos (1x1 grid)');
tos_regrid_ignore_NA.plot(ax=axs[2]) ; axs[2].set_title(im+' tos (1x1 grid; ignore NAs)');

### Special case: IPSL-CM6A-LR
If we wish to regrid using the full data of IPSL model, we will encounter the issue of NAs along 72.5E. The issue appears to stem from the weird longitude near 72.5E in this model (which is the longitude edge). To circumvent this, I think we need to either skip the first column (`lon[0]`) or the last column (`lon[-1]`), so that longitude can be "monotonic" across the edges. More info below:

In [ ]:
im = 'IPSL-CM6A-LR'
ds = read_data(im).squeeze()

In [ ]:
grid_in   = {'lon': ds['tos']['nav_lon'], 'lat': ds['tos']['nav_lat']} 
grid_out  = {'lon': np.linspace(0.5, 359.5, 360), 'lat': np.linspace(-89.5, 89.5, 180)}
regridder = xe.Regridder(grid_in, grid_out, 'bilinear', periodic=True, ignore_degenerate=True)
tos_regrid           = regridder(ds['tos'])
tos_regrid_ignore_NA = regridder(ds['tos'], skipna=True)

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(18,3))
axs      = axs.flatten()
ds['tos'].plot(ax=axs[0])            ; axs[0].set_title(im+' raw tos');
tos_regrid.plot(ax=axs[1])           ; axs[1].set_title(im+' tos (1x1 grid)');
tos_regrid_ignore_NA.plot(ax=axs[2]) ; axs[2].set_title(im+' tos (1x1 grid; ignore NAs)');

In [ ]:
grid_in   = {'lon': ds['tos']['nav_lon'][:,1:], 'lat': ds['tos']['nav_lat'][:,1:]} 
grid_out  = {'lon': np.linspace(0.5, 359.5, 360), 'lat': np.linspace(-89.5, 89.5, 180)}
regridder = xe.Regridder(grid_in, grid_out, 'bilinear', periodic=True, ignore_degenerate=True)
tos_regrid           = regridder(ds['tos'][:,1:])
tos_regrid_ignore_NA = regridder(ds['tos'][:,1:], skipna=True)

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=3,figsize=(18,3))
axs      = axs.flatten()
ds['tos'].plot(ax=axs[0])            ; axs[0].set_title(im+' raw tos');
tos_regrid.plot(ax=axs[1])           ; axs[1].set_title(im+' tos (1x1 grid; skip lon[0])');
tos_regrid_ignore_NA.plot(ax=axs[2]) ; axs[2].set_title(im+' tos (1x1 grid; skip lon[0]; ignore NAs)');

In [ ]:
ds['tos']['nav_lon'][100,:].values # IPSL weird longitude
# I think we should skip the first or the last one to make it monotonic through the edge, what do you think?